# Squaredle solver
Author: Aaron Patrick Monte (06/02/2023)

In [10]:
import twl
import networkx as nx
import math
from tqdm.notebook import tqdm
from IPython.core.magic import register_cell_magic

In [11]:
@register_cell_magic
def skip(line, cell):
    return

Each letter in the letters list can be connected horizontally, vertically, or diagonally in the 2D list.
Missing letters can be replaced by any non-alphabetical character.

Format is as follows: String 'asttaifnc' represents the board...
```
a s t
t a i
f n c
```

In [5]:
letters = '?st?ai?nc'
length = int(math.sqrt(len(letters)))


In [6]:
# Model a ixj board where each cell contains a letter as a directed graph
# where each cell is connected to its 8 neighbors (if they exist)
G = nx.DiGraph()

# Create nodes with letters
for i in range(length):
    for j in range(length):
        G.add_node((i, j), letter=letters[i*length+j])

# Create edges to connect each cell to its 8 neighbors
for i in range(length):
    for j in range(length):
        for x in range(i-1, i+2):
            for y in range(j-1, j+2):
                if (x, y) in G.nodes and (x, y) != (i, j):
                    G.add_edge((i, j), (x, y))

# Print all the letters in the board
for i in range(length):
    for j in range(length):
        print(G.nodes[(i, j)]['letter'], end=' ')
    print()

? s t 
? a i 
? n c 


## Brute-force approach

In [7]:
%%skip
# Print all words for node (0, 0). That is, find all connected combinations of nodes where each node cannot be visited more than once
words = []
combinations = {}
def find_words_brute_force(G, node, visited, word):
    visited[node] = True
    word += G.nodes[node]['letter']
    if len(word) >= 4 and twl.check(word) and word not in words:
        words.append(word)
        # Avoid iterating over G.nodes; use visited directly
        combinations[word] = [n for n, is_visited in visited.items() if is_visited]
    for neighbor in G.neighbors(node):
        if not visited[neighbor]:
            find_words_brute_force(G, neighbor, visited, word)
    visited[node] = False

UsageError: Cell magic `%%skip` not found.


In [5]:
%%skip
# Find all words for each node
visited = {node: False for node in G.nodes}
for node in tqdm(G.nodes):
    find_words_brute_force(G, node, visited, '')


## Informed search

In [6]:
# Informed search: Cross reference with the twl dictionary. If a node has an adjacent node with a letter that is not a prefix of any word in the dictionary, then it is not possible to form a word with that prefix
# This is a heuristic to prune the search space
words = []
combinations = {}
def find_words_informed_search(G, node, visited, word):
    visited[node] = True
    word += G.nodes[node]['letter']
    if len(word) >= 4 and twl.check(word) and word not in words:
        words.append(word)
        # Avoid iterating over G.nodes; use visited directly
        combinations[word] = [n for n, is_visited in visited.items() if is_visited]
    for neighbor in G.neighbors(node):
        if not visited[neighbor]:
            # Check if the neighbor has a letter that is not a prefix of any word in the dictionary
            neighbor_letter = G.nodes[neighbor]['letter']
            prefix = word + neighbor_letter
            if not any(twl.children(prefix)):
                continue
            find_words_informed_search(G, neighbor, visited, word)
    visited[node] = False

In [7]:
# Find all words for each node
visited = {node: False for node in G.nodes}
for node in tqdm(G.nodes):
    find_words_informed_search(G, node, visited, '')

  0%|          | 0/9 [00:00<?, ?it/s]

In [8]:
# Print all words and their combinations, sorted by length
for i in range(4, 17):
    # if words of length i do not exist, skip
    if not any([len(w) == i for w in words]):
        continue
    print(f'Words of length {i}:')
    for word in sorted([w for w in words if len(w) == i]):
        print(word, combinations[word])

Words of length 4:
aits [(0, 1), (0, 2), (1, 1), (1, 2)]
anis [(0, 1), (1, 1), (1, 2), (2, 1)]
cain [(1, 1), (1, 2), (2, 1), (2, 2)]
cast [(0, 1), (0, 2), (1, 1), (2, 2)]
cats [(0, 1), (0, 2), (1, 1), (2, 2)]
cist [(0, 1), (0, 2), (1, 2), (2, 2)]
nits [(0, 1), (0, 2), (1, 2), (2, 1)]
sain [(0, 1), (1, 1), (1, 2), (2, 1)]
sati [(0, 1), (0, 2), (1, 1), (1, 2)]
tain [(0, 2), (1, 1), (1, 2), (2, 1)]
Words of length 5:
satin [(0, 1), (0, 2), (1, 1), (1, 2), (2, 1)]
stain [(0, 1), (0, 2), (1, 1), (1, 2), (2, 1)]
Words of length 6:
nastic [(0, 1), (0, 2), (1, 1), (1, 2), (2, 1), (2, 2)]


In [ ]:
prefix = input('Enter a prefix: ')
length = int(input('Enter the length of the word: '))

if not any([len(w) == length and w.startswith(prefix) for w in words]):
    print('No words found')
elif len(prefix) < length:
    print(f'Words of length {length} starting with prefix {prefix} do not exist')
else:
    print(f'Words of length {length} starting with prefix {prefix}:')
    for word in sorted([w for w in words if w.startswith(prefix) and len(w) == length]):
        print(word, combinations[word])

No words found
